1. 데이터 load 

2. 데이터 결측값 처리

3. correlation (feature selection)

4. AutoML (model selection) base model + blender = 3+1, 5+1, 7+1 .....
                                     blender = catboost 많이 사용 2023.04 기준 

5. 선정된 모델들을 전부 다 optuna 로 최적의 hyper 를 찾음

6. stacking 에 모델들을 넣는다.
Ridge Classifier, Logistic Regression, Extra Trees Classifier

blender = catboost

In [3]:
import pandas as pd

f = pd.read_csv('titanic.csv')
f.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
f.Age.fillna(f.Age.median(), inplace=True)
# 여자:0 남자:1로 변환
f.Sex = f.Sex.map({'female':0, 'male':1})

# Embarked 누락값을 S로
f.Embarked.fillna('S', inplace=True)

embarked_dummies = pd.get_dummies(f.Embarked, prefix='Embarked', drop_first=True)

# 뒤에 붙여줌
titanic = pd.concat([f, embarked_dummies], axis=1)

In [7]:
f.corr() # Pclass, Sex, Fare	 

<ipython-input-7-46fba0b108b4>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  f.corr() # Pclass, Sex, Fare


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.034212,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.064910,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.339898,0.083081,0.018443,-0.549500
Age,0.034212,-0.064910,-0.339898,1.000000,-0.233296,-0.172482,0.096688
SibSp,-0.057527,-0.035322,0.083081,-0.233296,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.172482,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096688,0.159651,0.216225,1.000000


In [ ]:
!pip install pycaret

In [ ]:
# pycaret classification 모듈
from pycaret.classification import setup
# pycaret classification 모델들을 비교하기 위한 모듈
from pycaret.classification import compare_models
# 컬럼명 String으로 반환
# 데이터셋 설정
grid = setup(data=f, target="Survived",  verbose=True)
# 모델 학습
best = compare_models()
# best 모델 확인
print(best)

In [8]:
X = f[['Pclass', 'Sex', 'Fare']]
y = f.Survived

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2 , random_state=42)



In [ ]:
!pip install optuna

In [ ]:
import optuna
import numpy as np
import sklearn
import xgboost as xgb
def objective(trial):

    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.99),
        'n_estimators': trial.suggest_int("n_estimators", 1000, 10000, step=100),
      
    }


    model = xgb.XGBClassifier(**params, random_state = 7)

    bst = model.fit(X_train, y_train,eval_set=[(X_valid,y_valid)], early_stopping_rounds=50, verbose=False)
    preds = bst.predict(X_valid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_valid, pred_labels)
    return accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, show_progress_bar=True)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")


trial = study.best_trial


In [ ]:
from sklearn.ensemble import  StackingClassifier

# base_models 설정
base_models = [
                 ('rc', RidgeClassifier()),
                 ('lr', LogisticRegression(base_estimator=DecisionTreeClassifier()),
                  ('lr', ExtraTreesClassifier(base_estimator=DecisionTreeClassifier())
                ]

# stacking 설정
clf = StackingClassifier(estimators=base_models, final_estimator= CatboostClassfier() )

clf.fit(X_train,y_train)

In [ ]:
# 대회에 제출할 csv 파일을 생성